1. Inner Join과 Outer Join에 대해서 조사하고 이를 비교해봅시다.<br>
- 데이터베이스의 두 컬럼을 join할 때, inner join은 두 컬럼의 교집합을 구하는 연산이고 OUTER JOIN은 두 컬럼의 합집합을 구하는 연산이다. OUTER JOIN은 LEFT, RIGHT, FULL OUTER JOIN의 3가지로 나뉘는데 LEFT나 RIGHT는 각각 좌측, 우측 컬럼을 기준-시작으로 기준 컬럼과 기준 컬럼과 다른 컬럼의 교집합을 구하는 연산이다. 이 때 기준이 되는 컬럼에서부터 연산이 시작되는데 그 기준 컬럼을 DRIVE COLUMN이라 한다. FULL OUTER JOIN은 두 컬럼의 합집합을 구하는 연산이다.

2. Python의 set_trace기능을 살펴보고 이에 대한 예를 들어주세요 <br>
 - 파이썬 모듈에서 디버깅 보듈로 진입하는 기능이고, set_trace 부분에서 프로그램 실행을 중단함
 - pdb 모듈을 import

In [60]:
import pdb
 
def sum(x, y):
    z = x + y
    return z
 
a = 10
pdb.set_trace()  # 이곳에서 프로그램 중단
b = 20
c = sum(a, b)
print(c)

--Return--
None
> <ipython-input-60-4ac3081a9ca9>(8)<module>()
      6 
      7 a = 10
----> 8 pdb.set_trace()  # 이곳에서 프로그램 중단
      9 b = 20
     10 c = sum(a, b)


Documented commands (type help <topic>):
EOF    cl         debug    ignore    n       pp       run          unalias  
a      clear      disable  interact  next    psource  rv           undisplay
alias  commands   display  j         p       q        s            unt      
args   condition  down     jump      pdef    quit     skip_hidden  until    
b      cont       enable   l         pdoc    r        source       up       
break  context    exit     list      pfile   restart  step         w        
bt     continue   h        ll        pinfo   return   tbreak       whatis   
c      d          help     longlist  pinfo2  retval   u            where    

Miscellaneous help topics:
exec  pdb

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

> 

In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt


In [4]:
articles_df = (
  pd.read_csv('shared_articles.csv')
    .loc[lambda d: d['eventType'] == 'CONTENT SHARED']  # 공유된 컨텐츠만
)

# 처음부터 5개
articles_df.head(5)


,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [6]:
interaction_df = pd.read_csv('users_interactions.csv')

interaction_df.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [7]:
# dict type data
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

# event_type_strength 에 해당되는 사용자 데이터만
interaction_df['eventStrength'] = (
  interaction_df
    .loc[:, 'eventType']
    .apply(lambda d: event_type_strength[d])
)


In [9]:
interaction_df_over5 = (interaction_df
  .groupby('personId', group_keys=False)
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5]
)

# 인터렉션수 5 이상인 사용자의 데이터는 69868건
interaction_df_over5.shape[0]

69868

In [11]:
interaction_full_df = (
  interaction_df_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength']))
)

# pandas.core.frame.DataFrame의 데이터 interaction_full_df

interaction_full_df.head(10)    # 상위 10개

,personId,contentId,eventStrength,eventScore
0,-9223121837663643404,-8949113594875411859,1.0,1.000000
1,-9223121837663643404,-8377626164558006982,1.0,1.000000
2,-9223121837663643404,-8208801367848627943,1.0,1.000000
3,-9223121837663643404,-8187220755213888616,1.0,1.000000
4,-9223121837663643404,-7423191370472335463,8.0,3.169925
5,-9223121837663643404,-7331393944609614247,1.0,1.000000
6,-9223121837663643404,-6872546942144599345,1.0,1.000000
7,-9223121837663643404,-6728844082024523434,1.0,1.000000
8,-9223121837663643404,-6590819806697898649,1.0,1.000000
9,-9223121837663643404,-6558712014192834002,2.0,1.584963


In [12]:
interaction_train, interaction_test = train_test_split(
    interaction_full_df,
    stratify=interaction_full_df['personId'],
    test_size=0.2,
    random_state=42
)
# interactions on Train set: 31284
# interactions on Test set: 7822 
# 훈련집합은 전체 데이터의 20프로

In [13]:
# 평가 속도를 높이기 위해 personId를 기준으로 인덱스를 설정한다
interaction_full_indexed = interaction_full_df.set_index('personId')
interaction_train_indexed = interaction_train.set_index('personId')
interaction_test_indexed = interaction_test.set_index('personId')


In [14]:
def get_items_interacted(person_id, interaction_df):
    interated_items = interaction_df.loc[person_id]['contentId']
    return set(interated_items if type(interated_items) == pd.Series else [interated_items])


In [15]:
class ModelEvaluator:
    def __init__(self, n_non_interacted=100):
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = n_non_interacted
        
    def get_non_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interaction_full_indexed)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items
        
        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)
        
    def _verify_hit_top_n(self, item_id, recommend_items, topn):
        try:
            index = next(i for i, c in enumerate(recommend_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index
    
    def evaluate_model_for_user(self, model, person_id):
        interacted_values_testset = interaction_test_indexed.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
        
        interacted_items_count_testset = len(person_interacted_items_testset)
        
        # 특정 사용자에 대한 추천 순위 목록을 받아온다
        person_recs = model.recommend_items(
            person_id,
            items_to_ignore=get_items_interacted(person_id, interaction_train_indexed),
            topn=10000000000
        )
        
        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # test셋에서 사용자가 상호작용한 모든 항목에 대해 반복한다
        for item_id in person_interacted_items_testset:
            
            # 사용자가 상호작용하지 않은 100개 항목을 샘플링한다
            non_interacted_items_sample = self.get_non_interacted_items_sample(
                person_id,
                sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                seed=item_id % (2**32)
            )
            
            # 현재 선택한 item_id(상호작용 있었던 항목)와 100개 랜덤 샘플을 합친다
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))
            
            # 추천 결과물 중에서 현재 선택한 item_id와 100개 랜덤 샘플의 결과물로만 필터링한다
            valid_recs_df = person_recs[person_recs['contentId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['contentId'].values
            
            # 현재 선택한 item_id가 Top-N 추천 결과 안에 있는지 확인한다
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
            
        # Recall 값은 상호작용 있었던 항목들 중에서 관련없는 항목들과 섞였을 때 Top-N에 오른 항목들의 비율로 나타낼 수 있다
        recall_at_5 = hits_at_5_count / interacted_items_count_testset
        recall_at_10 = hits_at_10_count / interacted_items_count_testset
        
        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'interacted_count': interacted_items_count_testset,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10
        }
        return person_metrics
    
    def evaluate_model(self, model):
        people_metrics = []
        for idx, person_id in enumerate(list(interaction_test_indexed.index.unique().values)):
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)

        print('{} users processed'.format(idx))
        
        detailed_result = (
            pd.DataFrame(people_metrics)
              .sort_values('interacted_count', ascending=False)
        )
        
        global_recall_at_5 = detailed_result['hits@5_count'].sum() / detailed_result['interacted_count'].sum()
        global_recall_at_10 = detailed_result['hits@10_count'].sum() / detailed_result['interacted_count'].sum()
        
        global_metrics = {
            'model_name': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10
        }
        
        return global_metrics, detailed_result


In [16]:
# 객체 생성
model_evaluator = ModelEvaluator(n_non_interacted=100)

In [17]:
item_popularity = (interaction_full_df
 .groupby('contentId')['eventStrength'].sum()
 .sort_values(ascending=False)
 .reset_index()
)

# 인기도 모형
item_popularity.head(10)


,contentId,eventStrength
0,-4029704725707465084,457.5
1,-2358756719610361882,361.0
2,-6783772548752091658,357.0
3,8657408509986329668,338.0
4,-133139342397538859,336.5
5,-8208801367848627943,327.5
6,-6843047699859121724,315.0
7,2581138407738454418,310.0
8,2857117417189640073,308.5
9,-1633984990770981161,298.0


In [18]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 인기상품 중에서 사용자가 보지 않았던 상품을 추천한다
        recommendations = (
          self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)]
            .sort_values('eventStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['eventStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations


In [19]:
popularity_model = PopularityRecommender(item_popularity, articles_df)

In [20]:
print('Popularity 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(popularity_model)
print('Global Metrics:\n{}'.format(pop_global_metrics)) 
pop_detailed_results.head(10)

# Popularity 추천 모형을 평가합니다
# 1139 users processed
# Global Metrics:
# {'model_name': 'Popularity', 'recall@5': 0.22628483763743287, 'recall@10': 0.34671439529532089}


Popularity 추천 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Popularity', 'recall@5': 0.2241114804397852, 'recall@10': 0.3502940424443876}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,22,45,192,0.114583,0.234375,3609194402293569455
17,13,30,134,0.097015,0.223881,-2626634673110551643
16,14,32,130,0.107692,0.246154,-1032019229384696495
10,6,11,117,0.051282,0.094017,-1443636648652872475
82,24,35,88,0.272727,0.397727,-2979881261169775358
161,12,18,80,0.150000,0.225000,-3596626804281480007
65,15,26,73,0.205479,0.356164,1116121227607581999
81,14,19,69,0.202899,0.275362,692689608292948411
106,12,19,69,0.173913,0.275362,-9016528795238256703
52,20,29,68,0.294118,0.426471,3636910968448833585


In [21]:
# # stopwords 다운로드
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scv74\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [22]:
# stopwords를 제거한다
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

# 벡터의 길이는 5000, unigram과 bigram을 사용하고, stopwords를 제거하도록 학습한다
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=5000,
    stop_words=stopwords_list
)

item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['title'] + '' + articles_df['text'])
tfidf_feature_names = vectorizer.get_feature_names()

In [23]:
tfidf_matrix
# <3047x5000 sparse matrix of type '<class 'numpy.float64'>'
#   with 638928 stored elements in Compressed Sparse Row format>


<3047x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 638928 stored elements in Compressed Sparse Row format>

In [24]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id, interaction_indexed_df):
    interaction_person_df = interaction_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interaction_person_df['contentId'])
    
    user_item_strengths = np.array(interaction_person_df['eventStrength']).reshape(-1, 1)
    
    # 상호작용 강도를 바탕으로 가중치를 부여하여 평균 계산한다
    user_item_strengths_weighted_avg = \
        np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /\
        np.sum(user_item_strengths)
        
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_user_profiles():
    interaction_indexed_df = (interaction_full_df
        .loc[lambda d: d['contentId'].isin(articles_df['contentId'])]
        .set_index('personId')
    )
    user_profiles = {}
    
    for person_id in interaction_indexed_df.index.unique():
        user_profiles[person_id] = build_user_profile(person_id, interaction_indexed_df)
        
    return user_profiles


In [25]:
user_profiles = build_user_profiles()
len(user_profiles)
# 1140


1140

In [26]:
myprofile = user_profiles[-1479311724257856983].flatten().tolist()
pd.DataFrame(sorted(zip(tfidf_feature_names, myprofile), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])


,token,relevance
0,learning,0.312070
1,machine learning,0.269224
2,machine,0.256997
3,data,0.186630
4,google,0.171385
5,ai,0.141732
6,graph,0.114026
7,algorithms,0.109420
8,like,0.096113
9,language,0.085010


In [28]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, item_ids, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        # 유저 특성과 항목 특성 사이의 코사인 유사도를 구한다
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        
        # 가장 유사한 항목을 찾는다
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # 유사도를 기준으로 유사한 항목을 정렬한다
        similar_items = sorted(
            [(item_ids[i], cosine_similarities[0, i]) for i in similar_indices],
            key=lambda x: -x[1]
        )
        
        return similar_items
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        # 기존에 상호작용했던 항목은 제거한다
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations = (
            pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength'])
              .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations


In [30]:
content_based_model = ContentBasedRecommender(item_ids, articles_df)

content_based_model

In [31]:
print('콘텐츠 기반 추천 모형을 평가합니다')
cb_global_metrics, cb_detailed_results = model_evaluator.evaluate_model(content_based_model)
print('Global Metrics:\n{}'.format(cb_global_metrics))
cb_detailed_results.head(10)

# 콘텐츠 기반 추천 모형을 평가합니다
# 1139 users processed
# Global Metrics:
# {'model_name': 'Content-Based', 'recall@5': 0.38417284581948352, 'recall@10': 0.49411915111224752}


콘텐츠 기반 추천 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Content-Based', 'recall@5': 0.3834057785732549, 'recall@10': 0.4945026847353618}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,17,29,192,0.088542,0.151042,3609194402293569455
17,23,32,134,0.171642,0.238806,-2626634673110551643
16,21,35,130,0.161538,0.269231,-1032019229384696495
10,37,57,117,0.316239,0.487179,-1443636648652872475
82,8,15,88,0.090909,0.170455,-2979881261169775358
161,16,25,80,0.200000,0.312500,-3596626804281480007
65,10,16,73,0.136986,0.219178,1116121227607581999
81,11,20,69,0.159420,0.289855,692689608292948411
106,5,14,69,0.072464,0.202899,-9016528795238256703
52,5,10,68,0.073529,0.147059,3636910968448833585


In [39]:
# Matrix Factorization

users_items_pivot_df = (interaction_train
  .pivot(index='personId', columns='contentId', values='eventStrength')
  .fillna(0)
)


In [40]:
users_items_pivot_df.iloc[:5, :5]


contentId,-9222795471790223670,-9216926795620865886,-9194572880052200111,-9192549002213406534,-9190737901804729417
personId,,,,,
-9223121837663643404,0.0,0.0,0.0,0.0,0.0
-9212075797126931087,0.0,0.0,0.0,0.0,0.0
-9207251133131336884,0.0,3.0,0.0,0.0,0.0
-9199575329909162940,0.0,0.0,0.0,0.0,0.0
-9196668942822132778,0.0,0.0,0.0,0.0,0.0


In [46]:
users_items_pivot_matrix = users_items_pivot_df.values  # as_matrix()는 폐기됨, values 사용
users_items_pivot_matrix[:10]

# array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
#        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
#        [ 0.,  3.,  0., ...,  0.,  0.,  0.],
#        ..., 
#        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
#        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
#        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
user_ids = list(users_items_pivot_df.index)
user_ids[:10]

# [-9223121837663643404,
#  -9212075797126931087,
#  -9207251133131336884,
#  -9199575329909162940,
#  -9196668942822132778,
#  -9188188261933657343,
#  -9172914609055320039,
#  -9156344805277471150,
#  -9120685872592674274,
#  -9109785559521267180]


[-9223121837663643404,
 -9212075797126931087,
 -9207251133131336884,
 -9199575329909162940,
 -9196668942822132778,
 -9188188261933657343,
 -9172914609055320039,
 -9156344805277471150,
 -9120685872592674274,
 -9109785559521267180]

In [48]:
# User-Item matrix에서 요인의 개수를 정한다
NUMBER_OF_FACTORS_MF = 15

# User-Item Matrix을 분해한다
U, sigma, Vt = svds(users_items_pivot_matrix, k=NUMBER_OF_FACTORS_MF)


In [49]:
U.shape # (1140, 15)
Vt.shape # (15, 2926)

sigma_mat = np.diag(sigma)
sigma_mat.shape # (15, 15)


(15, 15)

In [50]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma_mat), Vt)
all_user_predicted_ratings

# array([[ -3.59736043e-03,   1.13569387e-03,   1.20882188e-02, ...,
#           1.92702938e-02,   5.12834501e-03,   2.51643553e-03],
#        [  9.71055720e-05,   8.75622910e-05,   3.93275556e-03, ...,
#           4.06961661e-04,  -4.19667479e-03,  -6.14562512e-04],
#        [ -2.52746201e-02,   4.07052544e-03,  -1.36013462e-02, ...,
#           1.51481593e-02,  -5.52722947e-03,   7.65220362e-03],
#        ..., 
#        [  2.07700031e-02,   1.02281302e-02,  -5.38400117e-02, ...,
#           2.87303379e-02,  -5.95440040e-02,  -3.52231861e-03],
#        [ -4.30766935e-02,   4.41158703e-03,   4.56109314e-02, ...,
#           2.21704728e-02,   5.25493080e-04,   1.05746293e-03],
#        [  1.55442912e-02,   2.01216659e-02,   3.37166289e-01, ...,
#           2.20774211e-03,   8.01526416e-02,   1.87568874e-02]])


array([[-3.59736043e-03,  1.13569387e-03,  1.20882188e-02, ...,
         1.92702938e-02,  5.12834501e-03,  2.51643553e-03],
       [ 9.71055720e-05,  8.75622910e-05,  3.93275556e-03, ...,
         4.06961661e-04, -4.19667479e-03, -6.14562512e-04],
       [-2.52746201e-02,  4.07052544e-03, -1.36013462e-02, ...,
         1.51481593e-02, -5.52722947e-03,  7.65220362e-03],
       ...,
       [ 2.07700031e-02,  1.02281302e-02, -5.38400117e-02, ...,
         2.87303379e-02, -5.95440040e-02, -3.52231861e-03],
       [-4.30766935e-02,  4.41158703e-03,  4.56109314e-02, ...,
         2.21704728e-02,  5.25493080e-04,  1.05746293e-03],
       [ 1.55442912e-02,  2.01216659e-02,  3.37166289e-01, ...,
         2.20774211e-03,  8.01526416e-02,  1.87568874e-02]])

In [51]:
# 재구성한 행렬을 pandas DataFrame으로 변환한다
cf_preds_df = (
  pd.DataFrame(all_user_predicted_ratings, 
               columns=users_items_pivot_df.columns, 
               index=user_ids)
    .transpose()
)


In [52]:
cf_preds_df.iloc[:5, :5]


,-9223121837663643404,-9212075797126931087,-9207251133131336884,-9199575329909162940,-9196668942822132778
contentId,,,,,
-9222795471790223670,-0.003597,0.000097,-0.025275,0.049606,-0.014388
-9216926795620865886,0.001136,0.000088,0.004071,-0.000690,0.001928
-9194572880052200111,0.012088,0.003933,-0.013601,-0.006128,0.035771
-9192549002213406534,0.028340,-0.003469,-0.025762,-0.006675,0.011599
-9190737901804729417,0.015525,-0.001208,0.008202,0.001825,-0.000400


In [53]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 사용자에 대한 예측값을 가져와서 정렬한다
        sorted_user_prediction = (self.cf_predictions_df
            .loc[:, user_id]
            .sort_values(ascending=False)
            .reset_index()
            .rename(columns={user_id: 'recStrength'})
        )
        
        recommendations = (sorted_user_prediction
            .loc[lambda d: ~d['contentId'].isin(items_to_ignore)]
            .sort_values('recStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.item_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations


In [54]:
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)


In [55]:
print('협업 필터링(SVD 행렬분해) 모형을 평가합니다')
cf_global_metrics, cf_detailed_results = model_evaluator.evaluate_model(cf_recommender_model)
print('Global Metrics:\n{}'.format(cf_global_metrics))
cf_detailed_results.head(10)

# 협업 필터링(SVD 행렬분해) 모형을 평가합니다
# 1139 users processed
# Global Metrics:
# {'model_name': 'Collaborative Filtering', 'recall@5': 0.26553311173612887, 'recall@10': 0.39798005625159805}


협업 필터링(SVD 행렬분해) 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Collaborative Filtering', 'recall@5': 0.2689849143441575, 'recall@10': 0.39682945538225517}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,22,42,192,0.114583,0.218750,3609194402293569455
17,22,35,134,0.164179,0.261194,-2626634673110551643
16,26,36,130,0.200000,0.276923,-1032019229384696495
10,33,48,117,0.282051,0.410256,-1443636648652872475
82,36,47,88,0.409091,0.534091,-2979881261169775358
161,17,28,80,0.212500,0.350000,-3596626804281480007
65,15,25,73,0.205479,0.342466,1116121227607581999
81,14,21,69,0.202899,0.304348,692689608292948411
106,16,24,69,0.231884,0.347826,-9016528795238256703
52,17,29,68,0.250000,0.426471,3636910968448833585


In [56]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 상위 1000개의 컨텐츠 기반 모형 추천을 가져온다
        cb_recs = (self.cb_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCB'})
        )
        
        # 상위 1000개의 협업필터링 추천을 가져온다
        cf_recs = (self.cf_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCF'})
        )
        
        # 1) 두 모형의 결과를 합친다
        # 2) CF, CB 모형의 점수를 바탕으로 Hybrid 모형의 점수를 계산한다
        # 3) Hybrid 점수를 기준으로 정렬한다
        recommendations = (cb_recs
            .merge(cf_recs, how='inner', left_on='contentId', right_on='contentId')
            .assign(recStrengthHybrid = lambda d: d['recStrengthCB'] * d['recStrengthCF'])
            .sort_values('recStrengthHybrid', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrengthHybrid', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations


In [57]:
hybrid_recommender_model = HybridRecommender(content_based_model, cf_recommender_model, articles_df)


In [58]:
print('하이브리드 모형을 평가합니다')
hybrid_global_metrics, hybrid_detailed_results = model_evaluator.evaluate_model(hybrid_recommender_model)
print('Global Metrics:\n{}'.format(hybrid_global_metrics))
hybrid_detailed_results.head(10)

# 하이브리드 모형을 평가합니다
# 1139 users processed
# Global Metrics:
# {'model_name': 'Hybrid', 'recall@5': 0.38340577857325492, 'recall@10': 0.49757095372027615}


하이브리드 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Hybrid', 'recall@5': 0.3825108667859882, 'recall@10': 0.49654819739197137}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,26,42,192,0.135417,0.218750,3609194402293569455
17,23,41,134,0.171642,0.305970,-2626634673110551643
16,26,38,130,0.200000,0.292308,-1032019229384696495
10,39,55,117,0.333333,0.470085,-1443636648652872475
82,26,32,88,0.295455,0.363636,-2979881261169775358
161,21,26,80,0.262500,0.325000,-3596626804281480007
65,14,23,73,0.191781,0.315068,1116121227607581999
81,13,20,69,0.188406,0.289855,692689608292948411
106,8,17,69,0.115942,0.246377,-9016528795238256703
52,14,22,68,0.205882,0.323529,3636910968448833585


In [59]:
pd.DataFrame([pop_global_metrics, cf_global_metrics, 
              cb_global_metrics, hybrid_global_metrics])


,model_name,recall@5,recall@10
0,Popularity,0.224111,0.350294
1,Collaborative Filtering,0.268985,0.396829
2,Content-Based,0.383406,0.494503
3,Hybrid,0.382511,0.496548
